In [1]:
# train model
from main_segmentation import trainModel
trainModel()

Epoch 1/20


ResourceExhaustedError: Graph execution error:

Detected at node 'gradient_tape/model/conv2d/Conv2D/Conv2DBackpropInput' defined at (most recent call last):
    File "c:\Users\SoulCRYSIS\anaconda3\lib\runpy.py", line 197, in _run_module_as_main
      return _run_code(code, main_globals, None,
    File "c:\Users\SoulCRYSIS\anaconda3\lib\runpy.py", line 87, in _run_code
      exec(code, run_globals)
    File "c:\Users\SoulCRYSIS\anaconda3\lib\site-packages\ipykernel_launcher.py", line 16, in <module>
      app.launch_new_instance()
    File "c:\Users\SoulCRYSIS\anaconda3\lib\site-packages\traitlets\config\application.py", line 846, in launch_instance
      app.start()
    File "c:\Users\SoulCRYSIS\anaconda3\lib\site-packages\ipykernel\kernelapp.py", line 677, in start
      self.io_loop.start()
    File "c:\Users\SoulCRYSIS\anaconda3\lib\site-packages\tornado\platform\asyncio.py", line 199, in start
      self.asyncio_loop.run_forever()
    File "c:\Users\SoulCRYSIS\anaconda3\lib\asyncio

In [4]:
# predict
from test import predictResults
predictResults()

--- select weight data ---
0 13-06-2022_400-400_250-20-5.hdf5
1 15-06-2022_400-400_250-20-3.hdf5
in_train_data/image234-1.jpg 800 800
1/1 [==============================] - 0s 162ms/step
in_train_data/image234-2.jpg 800 800
1/1 [==============================] - 0s 15ms/step
in_train_data/image235-1.jpg 800 800
1/1 [==============================] - 0s 17ms/step
in_train_data/image235-2.jpg 800 800
1/1 [==============================] - 0s 17ms/step
in_train_data/image236-0.jpg 800 800
1/1 [==============================] - 0s 16ms/step
not_train_data/image139-1.jpg 800 800
1/1 [==============================] - 0s 16ms/step
not_train_data/image145-1.jpg 800 800
1/1 [==============================] - 0s 16ms/step
not_train_data/image245-2.jpg 800 800
1/1 [==============================] - 0s 16ms/step
not_train_data/image247-1.jpg 800 800
1/1 [==============================] - 0s 15ms/step
not_train_data/image248-1.jpg 800 800
1/1 [==============================] - 0s 15ms/step


In [1]:
# train pair generator
import os
from constant import DATA_FOLDER, BATCH_SIZE
from constant import TRAIN_VALIDATOR_RATIO

# check train/test equality
images = os.listdir(DATA_FOLDER + "/train/images/")
labels = os.listdir(DATA_FOLDER + "/train/labels/")

isError = False
errorList = ["--- image that isn't in labels folder ---"]
for image in images:
    if image not in labels:
        errorList.append(image)
        isError = True
if len(errorList) > 1:
    for error in errorList:
        print(error)

errorList = ["--- label that isn't in images folder ---"]
for label in labels:
    if label not in images:
        errorList.append(label)
        isError = True
if len(errorList) > 1:
    for error in errorList:
        print(error)

if isError:
    raise Exception("images/labels folder didn't match")

# create train_pair
output = ""
toTrainSize = len(images)
while (
    round(toTrainSize * TRAIN_VALIDATOR_RATIO, 5) % (BATCH_SIZE * 2) != 0
    or round(toTrainSize * (1 - TRAIN_VALIDATOR_RATIO), 5) % (BATCH_SIZE * 2) != 0
):
    toTrainSize -= 1

for index, path in enumerate(images):
    if index >= toTrainSize:
        break
    output += "train/images/" + path
    output += " "
    output += "train/labels/" + path
    output += "\n"
print("created", toTrainSize, "pairs out of", len(images), "pairs")

with open(DATA_FOLDER + "/train_pair.lst", "w") as f:
    f.write(output)


created 240 pairs out of 268 pairs


In [ ]:
# resize image
import os
import glob
from constant import DATA_FOLDER
from PIL import Image

imagesPath = glob.glob("images/*")
NEW_SIZE = (800, 800)

def human_readable_size(size, decimal_places=2):
    for unit in ['B', 'KB', 'MB', 'GB', 'TB', 'PB']:
        if size < 1024.0 or unit == 'PB':
            break
        size /= 1024.0
    return f"{size:.{decimal_places}f} {unit}"

for path in imagesPath:
    img = Image.open(path)
    if img.size != NEW_SIZE:
        print(path, ":", img.size, "->", NEW_SIZE, ":", human_readable_size(os.path.getsize(path)), end=" -> ")
        img = img.resize(NEW_SIZE)
        img.save(path, quality=100)
        print(human_readable_size(os.path.getsize(path)))
